# Projeto: Séries temporais - MBA

Observações:

1. Para cada questão, crie novas células para efetuar seus códigos. 

2. Nomear variáveis de acordo com o contexto da questão. 

3. Caso necessário, utilize novas caixas de textos para deixar comentários. 

Prof. Jorge Luiz 

jorgearaujo@unifor.br

Você recebeu um dado chamado 'Electric_Production.csv'. Atribua-o a um dataframe df. 

1. A partir de df, use os modelos abaixo para fazer previsões para os 30 períodos futuros.


    a. ARIMA (2,0,0)

    b. ARIMA (2,0,1)
    
    c. ARIMA (2,2,1)
    
    d. ARIMA (8,2,1) 
    
    e. Criei um gráfico com todas as previsões acima. 
    
    f. Entre estes, qual é o melhor? Utilize a métrica AIC.  
    
    g. Use o auto-arima e apresente o melhor modelo encontrado. 